In [1]:
from PIL import Image, ImageDraw, ImageFont

In [2]:
class TextFont:
    '''
    This class is used to represent font family and font size for text.
    '''
    def __init__(self, font_family="simhei.ttf", font_size = 100):
        self.__font_family = font_family
        self.__font_size = font_size
    
    @property
    def font_family(self):
        return self.__font_family
    
    @font_family.setter
    def font_family(self, font_family):
        self.__font_family = font_family
    
    @property
    def font_size(self):
        return self.__font_size
    
    @font_size.setter
    def font_size(self, font_size):
        self.__font_size = font_size
    
    @property
    def font(self):
        return ImageFont.truetype(self.font_family, self.font_size)
    
    

In [3]:
def test_text_font():
    font = TextFont()
    print("Before assignment : ")
    print("font family : ", font.font_family)
    print("font size : ", font.font_size)
    print("font : ", font.font) 
#test_text_font()

In [37]:
class WaterMark:
    
    def add_text_to_image(self, image_path, text):
        img_to_process_rgba = self.__get_rgba_image(image_path)
        font_size = self.__get_font_size(img_to_process_rgba.size, text)
        font = TextFont()
        font.font_size = font_size
        fill = (50,50,50,50)
        text_overlay = self.__get_overlay(text, img_to_process_rgba.size, font.font, fill)
        img_added_watermark = Image.alpha_composite(img_to_process_rgba, text_overlay)
        return img_added_watermark 
    
    def __get_rgba_image(self, image_path):
        img_to_process = Image.open(image_path)
        return img_to_process.convert("RGBA")
    
    def __get_font_size(self, image_size, text):
        text_width = len(text)
        return int(min(image_size[0]/text_width, image_size[1], 100))
    
    def __get_overlay(self, text, img_size, font, fill):
        text_overlay = Image.new("RGBA", img_size, (255, 255, 255, 0))
        text_draw = ImageDraw.Draw(text_overlay)
        
        text_width, text_height = text_draw.textsize(text, font = font)
        for pos_x in range(0, img_size[0], text_width):
            for pos_y in range(0, img_size[1], text_height):
                text_draw.text((pos_x, pos_y), text, font = font, fill = fill)
        return text_overlay

In [42]:
water_mark = WaterMark()
img_processed = water_mark.add_text_to_image("./lena.jpg",'版权所有：Junnix')
img_processed.show()
img_processed.save("test.jpg")

In [34]:
print(min(2,5,1,6,-4))

-4
